# 영상 시각적 특징 분석

- 1배속 영상을 넣었을 때, 모델이 1, 1.1, 1.2, 1.3 배속 중 어떤 것 같은지 판단함

- Ex) 1배속 영상을 넣었는데, 1.3배속이라고 판단함 -> 이 영상의 특징은 무엇인가? (TI? SI? ...)

In [13]:
import cv2, yt_dlp, os, time, librosa, tempfile, uuid, glob, time, torch
from panns_inference import AudioTagging, SoundEventDetection, labels
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import soundfile as sf
from tqdm import tqdm
import torch.nn.functional as F
def extractFrames(cap, start_time_sec, duration_sec, fps, output_folder_path):
    """
    열려 있는 VideoCapture 스트림에서 특정 구간의 프레임을 추출하고 저장

    @param cap: cv2.VideoCapture 객체
    @param start_time_sec: 추출을 시작할 영상 지점(초)
    @param duration_sec: 추출할 길이(초)
    @param fps: 초당 추출할 프레임 수
    @param output_folrder_path: 이미지 저장 폴더
    """

    if not cap.isOpened():
        print(
            "오류: 유효하지 않은 VideoCapture 객체입니다. 스트림이 열려 있지 않습니다."
        )
        return False

    os.makedirs(output_folder_path)

    print(
        f"\n{start_time_sec}초부터 {duration_sec}초 동안 초당 {fps} 프레임 추출 시작..."
    )

    frame_count = 0
    last_saved_msec = start_time_sec * 1000  # 마지막으로 저장된 프레임의 시간 기준점
    target_frame_interval_msec = 1000 / fps

    # start_time_sec으로 이동(seek)
    # TODO 라이브 영상이면 seek이 안됨, 라이브면 함수가 동작하지 않도록 해야함
    cap.set(cv2.CAP_PROP_POS_MSEC, start_time_sec * 1000)

    # 실제 시작 시간 저장 (seek가 실패할 경우를 대비)
    actual_start_msec = cap.get(cv2.CAP_PROP_POS_MSEC)
    if (
        abs(actual_start_msec - (start_time_sec * 1000)) > 1000
    ):  # 1초 이상 차이나면 seek 실패로 간주
        print(
            f"경고: 스트림 탐색({start_time_sec}s)이 정확하지 않을 수 있습니다. (실제 시작: {actual_start_msec/1000.0:.2f}s)"
        )

    extraction_start_time = time.time()

    while 1:
        current_msec = cap.get(cv2.CAP_PROP_POS_MSEC)

        # 지정된 시간 구간을 벗어나면 추출 종료
        if current_msec / 1000.0 >= (start_time_sec + duration_sec):
            print(
                f"지정된 구간 ({start_time_sec}-{start_time_sec+duration_sec}초) 추출 완료."
            )
            break

        ret, frame = cap.read()

        if not ret:
            print(
                "스트림 끝에 도달했거나 더 이상 프레임을 읽을 수 없습니다. 추출 종료."
            )
            break

        # FPS에 맞추어 프레임 저장 여부 결정
        if frame_count == 0:
            filename = os.path.join(
                output_folder_path,
                f"{start_time_sec}s_{duration_sec}s_{fps}fps_{frame_count:03d}.jpg",
            )
            cv2.imwrite(filename, frame, [cv2.IMWRITE_JPEG_QUALITY, 80])
            frame_count += 1
            last_saved_msec = current_msec

        else:
            if (current_msec - last_saved_msec) >= target_frame_interval_msec:
                filename = os.path.join(
                    output_folder_path,
                    f"{start_time_sec}s_{duration_sec}s_{fps}fps_{frame_count:03d}.jpg",
                )
                cv2.imwrite(filename, frame, [cv2.IMWRITE_JPEG_QUALITY, 80])
                frame_count += 1
                last_saved_msec = current_msec

    extraction_end_time = time.time()

    print(
        f"총 {frame_count}개 프레임 추출 완료. 소요 시간: {extraction_end_time - extraction_start_time:.2f}초"
    )
    return frame_count

In [14]:
import utils
import os
import shutil

# 비디오 스트림 얻은 후 프레임 단위로 저장
cap = utils.openVideoStream('https://www.youtube.com/watch?v=h4ILpWwU1LM')
# utils.extractFrames720p(cap, 120, 10, 30, './frames')
extractFrames(cap, 120, 10, 30, './frames2')
# 프레임 저장 경로
src_dir = './frames2'
# 24씽 나눌 클립 폴더 경로
dst_root = './clips2'

os.makedirs(dst_root, exist_ok=True)

frame_files = sorted([
    f for f in os.listdir(src_dir) if f.lower().endswith('.jpg')    
])

# 24 프레임으로 모델 학습 -> 24장씩 잘라서 저장
clip_len = 24
num_clips = len(frame_files) // clip_len

for i in range(num_clips):
    clip_dir = os.path.join(dst_root, f"clip_{i:03d}")
    os.makedirs(clip_dir, exist_ok=True)

    for j in range(clip_len):
        frame_idx = i * clip_len + j
        src_path = os.path.join(src_dir, frame_files[frame_idx])
        dst_path = os.path.join(clip_dir, f"{j:03d}.jpg")
        shutil.copy2(src_path, dst_path)

    print(f"clip_{i:03d} 저장 완료")

'https://www.youtube.com/watch?v=h4ILpWwU1LM'에서 30fps 비디오 스트림 URL을 가져오는 중...
🎥 선택된 해상도: 590p @ 30fps
URL: https://rr2---sn-n3cgv5qc5oq-bh2sy.googlevideo.com/videoplayback?expire=1752676546&ei=YmR3aPeBM7KsvcAPgaGsiQs&ip=163.180.118.139&id=o-AK2MJLEM1U7oq2j5XHIhdYwbv28128Ij0aBaodeYXAZx&itag=136&aitags=133%2C134%2C135%2C136%2C160%2C242%2C243%2C244%2C247%2C278%2C298%2C299%2C302%2C303%2C308&source=youtube&requiressl=yes&xpc=EgVo2aDSNQ%3D%3D&met=1752654946%2C&mh=J2&mm=31%2C26&mn=sn-n3cgv5qc5oq-bh2sy%2Csn-oguesnd6&ms=au%2Conr&mv=m&mvi=2&pl=19&rms=au%2Cau&gcr=kr&initcwndbps=6530000&bui=AY1jyLPkjN-CfVcugKS1iCIqyyjPFK7acNqezSZ_JNc4NJzxJ3R_kLGFncaYCJfCXzq19PW88dqvtxq5&vprv=1&svpuc=1&mime=video%2Fmp4&ns=GoFW5_usUngLL1PDOhgBcJ4Q&rqh=1&gir=yes&clen=41029970&dur=299.566&lmt=1686381832025751&mt=1752654553&fvip=2&keepalive=yes&lmw=1&fexp=51544120&c=TVHTML5&sefc=1&txp=5432434&n=8PowRS7QPIu8Cw&sparams=expire%2Cei%2Cip%2Cid%2Caitags%2Csource%2Crequiressl%2Cxpc%2Cgcr%2Cbui%2Cvprv%2Csvpuc%2Cmime%2Cns%2Crqh%

In [ ]:
import torch
from torch import nn

# 하이퍼파라미터
N_CLASSES = 4  # [1.0, 1.1, 1.2, 1.3]

# MAC이라면 mps
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 구조 정의 (학습 때와 동일)
model = torch.hub.load("facebookresearch/pytorchvideo", "x3d_s", pretrained=True)
model.blocks[-1].proj = nn.Linear(2048, N_CLASSES)
model = model.to(DEVICE)

# 학습된 가중치 불러오기
model.load_state_dict(torch.load("checkpoints/best_ep9.pth", map_location=DEVICE))  # best_epX.pth는 저장된 파일명
model.eval()

In [ ]:
cap = utils.openVideoStream('https://www.youtube.com/watch?v=h4ILpWwU1LM')

import cv2, os, time

def extractFramesFast(cap, start_time_sec, duration_sec, fps, output_folder_path):
    """
    연속적으로 프레임을 읽으며 일정 간격마다 저장
    duration_sec이 None이면 start_time_sec부터 영상 끝까지 추출

    @param cap: cv2.VideoCapture 객체
    @param start_time_sec: 추출을 시작할 영상 지점(초)
    @param duration_sec: 추출할 길이(초)
    @param fps: 초당 추출할 프레임 수
    @param output_folder_path: 이미지 저장 폴더
    """

    if not cap.isOpened():
        print("❌ VideoCapture 열기 실패")
        return False

    os.makedirs(output_folder_path, exist_ok=True)

    # 영상 기본 정보
    video_fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    total_duration_sec = total_frames / video_fps

    # 시작/종료 프레임 계산
    start_frame = int(start_time_sec * video_fps)
    if duration_sec is None:
        end_frame = total_frames
        print(f"📌 duration_sec=None → 끝까지 추출 (총 {total_duration_sec:.2f}s)")
    else:
        end_frame = int((start_time_sec + duration_sec) * video_fps)

    frame_interval = max(1, int(round(video_fps / fps)))

    print(f"\n🎬 시작 프레임: {start_frame}, 종료 프레임: {end_frame}")
    print(f"🎯 프레임 간격: {frame_interval} (video fps: {video_fps:.2f})")

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    extraction_start_time = time.time()
    current_frame = start_frame
    saved_count = 0

    while current_frame < end_frame:
        ret, frame = cap.read()
        if not ret:
            print("❌ 프레임 읽기 실패 또는 영상 끝 도달")
            break

        if (current_frame - start_frame) % frame_interval == 0:
            filename = os.path.join(
                output_folder_path,
                f"{start_time_sec}s_{fps}fps_{saved_count:03d}.jpg"
                if duration_sec is None
                else f"{start_time_sec}s_{duration_sec}s_{fps}fps_{saved_count:03d}.jpg"
            )
            cv2.imwrite(filename, frame, [cv2.IMWRITE_JPEG_QUALITY, 80])
            saved_count += 1

        current_frame += 1

    extraction_end_time = time.time()
    print(
        f"✅ 총 {saved_count}개 프레임 저장 완료. 소요 시간: {extraction_end_time - extraction_start_time:.2f}초"
    )
    return saved_count




extractFramesFast(
    cap,
    start_time_sec=120,
    duration_sec=10,
    fps=cap.get(cv2.CAP_PROP_FPS),
    output_folder_path="./frames_30fps",
)
cap.release()

'https://www.youtube.com/watch?v=h4ILpWwU1LM'에서 30fps 비디오 스트림 URL을 가져오는 중...
🎥 선택된 해상도: 590p @ 30fps
URL: https://rr2---sn-n3cgv5qc5oq-bh2sy.googlevideo.com/videoplayback?expire=1752677240&ei=GGd3aL2TBpmavcAPvI_28Q0&ip=163.180.118.139&id=o-ADExVpyiDbYav5UAe3mMtiZcdf1S5BhQ9qzbLFrQLZih&itag=136&aitags=133%2C134%2C135%2C136%2C160%2C242%2C243%2C244%2C247%2C278%2C298%2C299%2C302%2C303%2C308&source=youtube&requiressl=yes&xpc=EgVo2aDSNQ%3D%3D&met=1752655640%2C&mh=J2&mm=31%2C26&mn=sn-n3cgv5qc5oq-bh2sy%2Csn-oguesnd6&ms=au%2Conr&mv=m&mvi=2&pl=19&rms=au%2Cau&gcr=kr&initcwndbps=3252500&bui=AY1jyLPUkRq0gt93qd2PkBifV2hdhrO0SWLaz2X22e2s8Xzk7IpqESR7dZOwImm4tkOvyESXu19DzmSQ&vprv=1&svpuc=1&mime=video%2Fmp4&ns=_EVB6_xol7RFVp91lueKRv8Q&rqh=1&gir=yes&clen=41029970&dur=299.566&lmt=1686381832025751&mt=1752655282&fvip=2&keepalive=yes&lmw=1&fexp=51544120&c=TVHTML5&sefc=1&txp=5432434&n=l1SSoilFBRfV0A&sparams=expire%2Cei%2Cip%2Cid%2Caitags%2Csource%2Crequiressl%2Cxpc%2Cgcr%2Cbui%2Cvprv%2Csvpuc%2Cmime%2Cns%2Crqh%